<a href="https://colab.research.google.com/github/autumnjohnson/ceti_audio/blob/main/load_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/autumnjohnson/ceti_audio.git

In [ ]:
import IPython.display as ipd
from collections import Counter
import io
import pandas as pd
from librosa.util import find_files
from datasets import Dataset, Value, Audio, Features, GeneratorBasedBuilder, BuilderConfig, Version, DatasetInfo, load_dataset
import pyarrow as pa 
import pathlib
from huggingface_hub import login
import soundfile as sf
